In [1]:
import math
from collections import defaultdict


In [22]:
edge = 8
n = edge**2
num_proc = 7

# Method 3

In [23]:
part2proc = {}
proc2par = defaultdict(set)
neighbors = {}
exchanges = defaultdict(set)

## 1. Determine the neighbor of each particle

In [24]:
for i in range(edge):
    for j in range(edge):
        particle = edge * i + j

        right = j + 1
        if right >= edge:
            right = 0
        neigh_right = edge * i + right

        left = j - 1
        if left < 0:
            left = edge - 1
        neigh_left = edge * i + left

        top = i - 1
        if top < 0:
            top = edge - 1
        neigh_top = edge * top + j

        bot = i + 1
        if bot >= edge:
            bot = 0
        neigh_bot = edge * bot + j

        neighbors[particle] = {
            "top": neigh_top,
            "left": neigh_left,
            "bottom": neigh_bot,
            "right": neigh_right
        }

## 2. Determine to which process the particle belongs

In [25]:
split = math.ceil(n / num_proc)
for i in range(n):
    part2proc[i] = i // split
    proc2par[i // split].add(i)

In [26]:
for proc in range(num_proc):
    particles = proc2par[proc]
    for p in particles:
        for neighbor in neighbors[p].values():
            p_exchange = part2proc[neighbor]
            if p_exchange != proc:
                exchanges[proc].add(p_exchange)


## 3. Measure the metrics

### 3.1. Average difference in num. particles

In [27]:
diffs = []

for p1 in range(num_proc):
    for p2 in range(p1 + 1, num_proc):
        diffs.append(abs(len(proc2par[p1]) - len(proc2par[p2])))

mean_diff = sum(diffs) / len(diffs)

mean_diff

1.7142857142857142

### 3.2. Average out communication

In [28]:
comms = []

for exchange in exchanges.values():
    comms.append(len(exchange))

mean_comm = sum(comms) / len(comms)

mean_comm

2.2857142857142856

# Method 2

In [29]:
part2proc = {}
proc2par = defaultdict(set)
neighbors = {}
exchanges = defaultdict(set)

## 1. Determine the neighbor of each particle

In [30]:
for i in range(edge):
    for j in range(edge):
        particle = edge * i + j

        right = j + 1
        if right >= edge:
            right = 0
        neigh_right = edge * i + right

        left = j - 1
        if left < 0:
            left = edge - 1
        neigh_left = edge * i + left

        top = i - 1
        if top < 0:
            top = edge - 1
        neigh_top = edge * top + j

        bot = i + 1
        if bot >= edge:
            bot = 0
        neigh_bot = edge * bot + j

        neighbors[particle] = {
            "top": neigh_top,
            "left": neigh_left,
            "bottom": neigh_bot,
            "right": neigh_right
        }

## 2. Determine to which process the particle belongs

In [31]:
split = math.floor(edge / num_proc)

for i in range(edge):
    for j in range(edge):
        particle = i * edge + j

        if split > 0:
            proc = min(i // split, num_proc - 1)
        else:
            proc = -1
        part2proc[particle] = proc
        proc2par[proc].add(particle)

In [32]:
for proc in range(num_proc):
    particles = proc2par[proc]
    for p in particles:
        for neighbor in neighbors[p].values():
            p_exchange = part2proc[neighbor]
            if p_exchange != proc:
                exchanges[proc].add(p_exchange)


## 3. Measure the metrics

### 3.1. Average difference in num. particles

In [33]:
if split == 0:
    mean_diff = -1
else:
    diffs = []

    for p1 in range(num_proc):
        for p2 in range(p1 + 1, num_proc):
            diffs.append(abs(len(proc2par[p1]) - len(proc2par[p2])))

    mean_diff = sum(diffs) / len(diffs)

mean_diff

2.2857142857142856

### 3.2. Average out communication

In [34]:
if split == 0:
    mean_comm = -1
else:
    comms = []

    for exchange in exchanges.values():
        comms.append(len(exchange))

    mean_comm = sum(comms) / len(comms)

mean_comm

2.0

# Method 1

In [35]:
part2proc = {}
proc2par = defaultdict(set)
neighbors = {}
exchanges = defaultdict(set)

## 1. Determine the neighbor of each particle

In [36]:
for i in range(edge):
    for j in range(edge):
        particle = edge * i + j

        right = j + 1
        if right >= edge:
            right = 0
        neigh_right = edge * i + right

        left = j - 1
        if left < 0:
            left = edge - 1
        neigh_left = edge * i + left

        top = i - 1
        if top < 0:
            top = edge - 1
        neigh_top = edge * top + j

        bot = i + 1
        if bot >= edge:
            bot = 0
        neigh_bot = edge * bot + j

        neighbors[particle] = {
            "top": neigh_top,
            "left": neigh_left,
            "bottom": neigh_bot,
            "right": neigh_right
        }

## 2. Determine to which process the particle belongs

In [37]:
def find_ab(n: int):
    def _find_ab(n: int):
        a = math.floor(math.sqrt(n))
        while math.floor(n / a) * a != n and a > 1:
            a -= 1

        b = n // a

        return a, b

    if n % 2 != 0:
        a, b = _find_ab(n)
        if a != 1:
            return a, b, False
        else:
            return *_find_ab(n - 1), True
    else:
        return *_find_ab(n), False

a, b, is_redundant = find_ab(num_proc)

if is_redundant is True:
    split_i, split_j = math.ceil(edge / (a + 1)), math.ceil(edge / b)
else:
    split_i, split_j = math.ceil(edge / a), math.ceil(edge / b)

# Normal processes
for i in range(edge):
    for j in range(edge):
        particle = i * edge + j

        if i // split_i >= a:
            # In odd process region
            proc = num_proc - 1
        else:
            j_proc = j // split_j
            i_proc = i // split_i
            proc = i_proc * (a + 1) + j_proc

        part2proc[particle] = proc
        proc2par[proc].add(particle)

In [38]:
for i in range(edge):
    for j in range(edge):
        parID = i * edge + j
        proc = part2proc[parID]

        print(f"{proc:3d}", end='')
        if j == edge - 1:
            print()

  0  0  0  1  1  1  2  2
  0  0  0  1  1  1  2  2
  0  0  0  1  1  1  2  2
  3  3  3  4  4  4  5  5
  3  3  3  4  4  4  5  5
  3  3  3  4  4  4  5  5
  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6


In [39]:
for proc in range(num_proc):
    particles = proc2par[proc]
    for p in particles:
        for neighbor in neighbors[p].values():
            p_exchange = part2proc[neighbor]
            if p_exchange != proc:
                exchanges[proc].add(p_exchange)


## 3. Measure the metrics

### 3.1. Average difference in num. particles

In [40]:
diffs = []

for p1 in range(num_proc):
    for p2 in range(p1 + 1, num_proc):
        diffs.append(abs(len(proc2par[p1]) - len(proc2par[p2])))

mean_diff = sum(diffs) / len(diffs)

mean_diff

3.4285714285714284

### 3.2. Average out communication

In [41]:
comms = []

for exchange in exchanges.values():
    comms.append(len(exchange))

mean_comm = sum(comms) / len(comms)

mean_comm

4.285714285714286